# **TESTING FOR TRANSFORMERS**

The Transformers unit tests are divided into 2 separate documents. The reason for this is that the storage capacity of Google Colab does not allow (without payment options) to run the code for all pre-trained models.
Therefore, it has been decided to split the pre-trained models:
- **Part 1**: BERT / BERT LARGE / ROBERTA / DISTILBERT
- **Part 2**: ALBERT / ALBERT XXLARGE / DEBERTA

### IMPORT PACKAGES

In [ ]:
!pip install transformers
!pip install simpletransformers #specific for ALBERT transformer
import transformers
from transformers import TFAutoModel
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Dropout, Dense, Bidirectional, LSTM, GRU
from keras import Model, optimizers, callbacks
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pydotplus
import keras
import re
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords  # stopwords
from nltk import word_tokenize, sent_tokenize  # tokenizing
from nltk.stem import PorterStemmer, LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## DOWNLOAD PRETRAINED MODELS FROM HUGGINGFACE
For each model:

* Tokenizer
* Pretrained model



In [ ]:
pre_models = {
    "ALBERT": ( transformers.AlbertTokenizer.from_pretrained('albert-base-v2'),
               TFAutoModel.from_pretrained('albert-base-v2', from_pt=True)
               ),
    "ALBERT XXLARGE":( transformers.AlbertTokenizer.from_pretrained('albert-xxlarge-v2'), 
                        TFAutoModel.from_pretrained('albert-xxlarge-v2', from_pt=True)
                        ),
    "DEBERTA": ( transformers.DebertaTokenizer.from_pretrained('microsoft/deberta-base'),
                TFAutoModel.from_pretrained('microsoft/deberta-base', from_pt=True)
                )
}

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.dense.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaModel: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing TFDebertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDebertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaModel for predictions wi

In [ ]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-z_+\-*]', ' ', text).strip()
    lemm = WordNetLemmatizer()
    title = []
    for token in text.split():
        title.append(token)
    return ' '.join(str(elem) for elem in title)

In [ ]:
def encode(text, tokenizer, max_len):
    code = tokenizer.batch_encode_plus(batch_text_or_text_pairs=text,
                                       max_length=max_len,
                                       return_tensors='pt',
                                       return_token_type_ids=False,
                                       truncation=True,
                                       padding='max_length')
    return np.array(code['input_ids'])

## IMPORT DATA FROM FILES
- Sample provided is splitted into train and test by a default value of 25%

In [ ]:
uploaded = files.upload()
data = pd.read_excel('/content/sample_400_Data&Headers.xlsx')
train_data, test_data, train_label, test_label = train_test_split(data['headline'],data['labels'] ,random_state=104,test_size=0.25, shuffle=True)

Saving sample_400_Data&Headers.xlsx to sample_400_Data&Headers.xlsx


## DATA PREPARATION

In [ ]:
train_data = train_data.apply(lambda x: preprocessing(str(x)).lower()) 
test_data = test_data.apply(lambda x: preprocessing(str(x)).lower())

In [ ]:
max_len = 20

In [ ]:
label_encoder = LabelEncoder()
num_label = train_label.nunique()
train_label_transformed = label_encoder.fit_transform(train_label)
test_label_transformed = label_encoder.fit_transform(test_label)
train_label_encoded = to_categorical(train_label_transformed, num_classes=num_label, dtype='int32')
test_label_encoded = to_categorical(test_label_transformed, num_classes=num_label, dtype='int32')

In [ ]:
def build_model(transformer, loss='categorical_crossentropy', max_len=max_len):
    input_word_ids = Input(shape=(max_len,), dtype='int32', name="input_word_ids")
    print(input_word_ids)
    sequence_output = transformer(input_word_ids)[0]
    print(sequence_output)
    drop = Dropout(0.5, name='Dropout_1')(sequence_output)
    lstm = Bidirectional(LSTM(100, name='LSTM'))(drop)
    layer = Dropout(0.5, name='Dropout_2')(lstm)
    out = Dense(num_label, name='Dense')(layer)

    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(optimizer=tf.optimizers.Adam(), loss=loss, metrics=['accuracy'])
    return model

In [ ]:
BATCH_SIZE = 32
num_epochs = 4

In [ ]:
def create_datasets(train_data_encoded, train_label_encoded, test_data_encoded):
  AUTO = tf.data.experimental.AUTOTUNE
  train_dataset = (
      tf.data.Dataset
      .from_tensor_slices((train_data_encoded, tf.convert_to_tensor(train_label_encoded)))
      .repeat()
      .shuffle(2048)
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
  )
  test_dataset = (
      tf.data.Dataset
      .from_tensor_slices(test_data_encoded)
      .batch(BATCH_SIZE)
  )
  n_steps = train_data.shape[0] // BATCH_SIZE
  return train_dataset, test_dataset, n_steps

##Setting test parameters & executing test

In [ ]:
import unittest
class Test(unittest.TestCase):
  def __init__(self, testName, output):
    super(Test, self).__init__(testName) 
    self.output = output

  def test_type_prediction(self):
    error_message = "Output of model is not a dinamic array"
    self.assertEqual(self.output, np.ndarray, error_message)
  
  def test_length_predictions(self):
    error_message = "The length of the prediction is not the expected"
    self.assertEqual(self.output, 100, error_message)
  
  def test_ANN_structure(self):
    layers = self.output.layers
    layer_structure = list()
    for layer in layers:
      if str(type(layer)).__contains__('transformers.models'):
        layer_structure.append('transformers.models')
      else:
        layer_structure.append(type(layer))

    lista_layers = [
        keras.engine.input_layer.InputLayer,
        'transformers.models',
        keras.layers.regularization.dropout.Dropout,
        keras.layers.rnn.bidirectional.Bidirectional,
        keras.layers.regularization.dropout.Dropout,
        keras.layers.core.dense.Dense
    ]
    error_message = "The ANN structure is not the expected"
    self.assertEqual(layer_structure, lista_layers, error_message)

In [ ]:
output = []
models = []
for pre_model in pre_models:
  tokenizer = pre_models[pre_model][0]
  transformer = pre_models[pre_model][1]
  model = build_model(transformer, max_len=max_len)
  train_data_encoded = encode(train_data.astype('str'), tokenizer, max_len=max_len)
  test_data_encoded = encode(test_data.astype('str'), tokenizer, max_len=max_len)
  train_dataset, test_dataset, n_steps = create_datasets(train_data_encoded, train_label_encoded, test_data_encoded)

  model.fit(train_dataset, steps_per_epoch=n_steps, epochs=num_epochs)
  output.append((type(model.predict(test_data_encoded)), len(model.predict(test_data_encoded))))
  models.append(model)

KerasTensor(type_spec=TensorSpec(shape=(None, 20), dtype=tf.int32, name='input_word_ids'), name='input_word_ids', description="created by layer 'input_word_ids'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 768), dtype=tf.float32, name=None), name='tf_albert_model/albert/encoder/albert_layer_groups_._0/albert_layers_._0/full_layer_layer_norm/batchnorm_11/add_1:0', description="created by layer 'tf_albert_model'")
Epoch 1/4


9/9 [==============================] - 23s 134ms/step - loss: 4.3354 - accuracy: 0.2500
Epoch 2/4
9/9 [==============================] - 1s 132ms/step - loss: 1.6796 - accuracy: 0.2743
Epoch 3/4
9/9 [==============================] - 1s 135ms/step - loss: 1.4039 - accuracy: 0.2535
Epoch 4/4
4/4 [==============================] - 0s 55ms/step
KerasTensor(type_spec=TensorSpec(shape=(None, 20), dtype=tf.int32, name='input_word_ids'), name='input_word_ids', description="created by layer 'input_word_ids'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 4096), dtype=tf.float32, name=None), name='tf_albert_model_1/albert/encoder/albert_layer_groups_._0/albert_layers_._0/full_layer_layer_norm/batchnorm_11/add_1:0', description="created by layer 'tf_albert_model_1'")
Epoch 1/4


9/9 [==============================] - 36s 2s/step - loss: 4.2116 - accuracy: 0.2396
Epoch 2/4
9/9 [==============================] - 23s 3s/step - loss: 1.4226 - accuracy: 0.2500
Epoch 3/4
9/9 [==============================] - 24s 3s/step - loss: 1.4181 - accuracy: 0.2917
Epoch 4/4
4/4 [==============================] - 3s 606ms/step
KerasTensor(type_spec=TensorSpec(shape=(None, 20), dtype=tf.int32, name='input_word_ids'), name='input_word_ids', description="created by layer 'input_word_ids'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 768), dtype=tf.float32, name=None), name='tf_deberta_model/deberta/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_deberta_model'")
Epoch 1/4


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


9/9 [==============================] - 37s 234ms/step - loss: 3.7946 - accuracy: 0.2604
Epoch 2/4
9/9 [==============================] - 2s 233ms/step - loss: 1.9919 - accuracy: 0.2222
Epoch 3/4
9/9 [==============================] - 2s 227ms/step - loss: 5.9047 - accuracy: 0.2951
Epoch 4/4
4/4 [==============================] - 0s 62ms/step


In [ ]:
suite = unittest.TestSuite()
output_list = output
model_list = models
for i, model in enumerate(pre_models):
  (output_type, output_len) = output_list[i]
  suite.addTest(Test('test_type_prediction', output_type))
  suite.addTest(Test('test_length_predictions', output_len))
  suite.addTest(Test('test_ANN_structure', model_list[i]))
unittest.TextTestRunner(verbosity=3).run(suite)

test_type_prediction (__main__.Test) ... ok
test_length_predictions (__main__.Test) ... ok
test_ANN_structure (__main__.Test) ... ok
test_type_prediction (__main__.Test) ... ok
test_length_predictions (__main__.Test) ... ok
test_ANN_structure (__main__.Test) ... ok
test_type_prediction (__main__.Test) ... ok
test_length_predictions (__main__.Test) ... ok
test_ANN_structure (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.025s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>